In [8]:
import pickle
import re
import string
import joblib

In [9]:
# Muat model dan vectorizer dari file
svm = joblib.load('/content/svm_tfidf_model.pkl')
tfidf_vectorizer = joblib.load('/content/tfidf_vectorizer.pkl')


In [10]:
new_text = [
    "Aplikasi ini sangat bagus dan berkualitas tinggi!"
]


In [11]:

# Muat slang words
fSlang = "/content/new_kamusalay.csv"
sw = open(fSlang, encoding='utf-8', errors='ignore', mode='r')
SlangS = sw.readlines()
sw.close()
SlangS = {slang.strip().split(',')[0]: slang.strip().split(',')[1] for slang in SlangS}

# Muat stopwords
fStop = "/content/stopwordbahasa.csv"
st = open(fStop, encoding='utf-8', errors='ignore', mode='r')
StopS = st.readlines()
st.close()
StopS = {Stop.strip() for Stop in StopS}

# Load the models (ubah ekstensi file menjadi .pkl)
svm_model = pickle.load(open('svm_tfidf_model.pkl', 'rb'))
tfidf = pickle.load(open('tfidf_vectorizer.pkl', 'rb'))
le = pickle.load(open('label_encoder.pkl', 'rb'))

# Fungsi untuk slang words dan stopwords
def formaldanstop(t):
    t = t.split()
    for i, x in enumerate(t):
        if x in SlangS.keys():
            t[i] = SlangS[x]
    return ' '.join(x for x in t if x not in StopS)

# Fungsi untuk remove duplicate letters
def remove_duplicate_letters(text):
    return re.sub(r'(.)\1+', r'\1', text)

# Fungsi untuk memprediksi sentiment
def predict_sentiment(new_text):
    # Preprocessing
    new_text = new_text.lower()  # Case folding
    new_text = re.sub(r'\d+', '', new_text)  # Remove numbers
    new_text = re.sub(r'[^\w\s]', '', new_text)  # Remove punctuation

    # Gunakan slang word dan stopword removal
    new_text = formaldanstop(new_text)

    # Gunakan function to remove duplicate letters
    new_text = remove_duplicate_letters(new_text)

    # Membuat TF-IDF features
    new_text_features = tfidf.transform([new_text])

    # Membuat prediksi
    prediction = svm_model.predict(new_text_features)

    # Sentimen label
    sentiment_label = le.inverse_transform(prediction)[0]

    return sentiment_label



In [12]:
# Contoh Penggunaan
new_text = "Aplikasi ini sangat bagus dan berkualitas tinggi!"
predicted_sentiment = predict_sentiment(new_text)
print(f"Sentiment for '{new_text}': {predicted_sentiment}")

Sentiment for 'Aplikasi ini sangat bagus dan berkualitas tinggi!': Positif
